# Modeling drug effects

In this notebook we will try to model drug effects, and we will focus on a few of the so called [CIPA drugs](https://cipaproject.org/wp-content/uploads/2016/05/CiPA-Compounds.pdf). These are characterised into *high*, *intermediate* and *low* risk.

The way we model drug effects is to change some parameters in the model.

We will use a libarary called [`drug-database`](https://computationalphysiology.github.io/drug-database/README.html) to get the scaling factors of the different drugs. This is a very simple library that just returns the scaling factors as a dictionary. Let us start by just listing the available drugs

In [ ]:
import pprint
import drug_database
factors = drug_database.get_drug_factors()

pprint.pprint(f"Drugs: {list(factors.keys())}")

Now let us focus our attention on a specific drug. Let us choose *Dofetilide* which is a high risk drug and print the factors:

In [ ]:
factors["Dofetilide"]

Here we get three different dictionaries, with keys `1`, `5` and `10`. The different numbers indicate the strength of the dose; higher values means higher dose. Note that useually the dose of a drug is measured molar concentration, but these numbers are normalized in a way that makes it easier to compare different drugs.

Let us pick the factors for the dose corresponding to the value `1`, and plot them as a bar plot

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots()
ax.bar(factors["Dofetilide"]['1'].keys(), factors["Dofetilide"]['1'].values())
ax.set_xticklabels(factors["Dofetilide"]['1'].keys(), rotation=30)
ax.grid()
plt.show()

If one scaling factors is equal to 1.0 it means that the corresponding ion channel is not affected by the drug. I a value if smaller than 1.0 it means that the channel the blocked by the drug. In this case we see that this drug blocks the L-type Calcium channel.

Let us see how we solve the model using these scaling factors. Let us first solve the model without any drug. This model will be reffered to as the control

In [ ]:
import utils

In [ ]:
model_name = "ORdmm_Land"
model = utils.load_model(model_name)

We will solve the model for 10 beats

In [ ]:
num_beats = 10
dt = 0.02
t_start = 0.0
t_end = 1000.0

In [ ]:
u0 = model.initial_state_values()
for i in range(num_beats):
    control_solution = model.solve(t_start=t_start, t_end=t_end, dt=dt, u0=u0)
    u0 = control_solution.u[-1, :]

Now, we will apply the drug parameters

In [ ]:
parameters = model.default_parameters()
parameters.update(factors["Dofetilide"]['1'])
pprint.pprint(parameters)

In [ ]:
u0 = model.initial_state_values()
for i in range(num_beats):
    drug_solution = model.solve(t_start=t_start, t_end=t_end, dt=dt, u0=u0, parameters=parameters)
    u0 = drug_solution.u[-1, :]

Now let us plot the corresponding voltage, intracellular calcium concentration and the current accross the L-type calcium channel

In [ ]:
fig, ax = plt.subplots(3, 1, sharex=True, figsize=(8, 8))
ax[0].plot(control_solution.time, control_solution["v"], label="control")
ax[1].plot(control_solution.time, control_solution["cai"], label="control")
ax[2].plot(control_solution.time, control_solution.monitored("ICaL"), label="control")
ax[0].plot(drug_solution.time, drug_solution["v"], label="drug")
ax[1].plot(drug_solution.time, drug_solution["cai"], label="drug")
ax[2].plot(drug_solution.time, drug_solution.monitored("ICaL"), label="drug")

ax[0].set_ylabel("Volutage [mV]")
ax[1].set_ylabel("Caclium [mM]")
ax[2].set_ylabel("ICaL")
ax[2].set_xlabel("ms")
ax[0].legend()
plt.show()

## Excercise

Compare the traces using different doses of the drug `Dofetilide`. How does this change the resulting traces?

### Double click for solution

<!--

results = {"control": control_solution}

for dose_name, dose_factors in factors["Dofetilide"].items():
    parameters = model.default_parameters()
    parameters.update(dose_factors)
    
    u0 = model.initial_state_values()
    for i in range(num_beats):
        results[dose_name] = model.solve(t_start=t_start, t_end=t_end, dt=dt, u0=u0, parameters=parameters)
        u0 = results[dose_name].u[-1, :]
    
fig, ax = plt.subplots(3, 1, sharex=True, figsize=(8, 8))
for label, solution in results.items():
    ax[0].plot(solution.time, solution["v"], label=label)
    ax[1].plot(solution.time, solution["cai"], label=label)
    ax[2].plot(solution.time, solution.monitored("ICaL"), label=label)


ax[0].set_ylabel("Volutage [mV]")
ax[1].set_ylabel("Caclium [mM]")
ax[2].set_ylabel("ICaL")
ax[2].set_xlabel("ms")
ax[0].legend()
plt.show()

-->